In [24]:
import os
import openai
from openai import OpenAI
import pandas as pd
from tqdm.contrib.concurrent import thread_map
import time
from multiprocessing import Pool
from tqdm import tqdm
from tqdm import tqdm_notebook
import numpy as np
import string
from bs4 import BeautifulSoup
import unicodedata
import re
from sklearn.metrics import classification_report

pd.set_option('display.max_colwidth', 1)

In [25]:
client = OpenAI(api_key="")

In [71]:
def classifica_com_gpt(text):
    MODEL="gpt-4-turbo"
    
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": "Você é um classificador de ironias, analise o texto abaixo, que veio do contexto de redes sociais. Responda apenas com 'ironia' se o texto conter ironia ou 'n_ironia' se o texto não conter ironia"},
        {"role": "user", "content": text}
      ]
    )
    return completion.choices[0].message.content

In [72]:
classifica_com_gpt('oi')

'n_ironia'

In [73]:
data = pd.read_csv('data_ironia/teste_ironias_reduzido.csv', sep='\t', lineterminator='\n')

In [74]:
len(data)

1500

In [75]:
data = data[['text', 'prediction']]

In [76]:
data.head(2)

,text,prediction
0,Aquele momento que o gato preto volta pra não cruzar o seu caminho,1
1,"Marcelo Oliveira pode estar poupando Matheus Sales para encarar o Santos, de Lucas Lima.",1


In [77]:
tqdm.pandas(desc="Classificando textos")
data['pred_gpt4_turbo'] = data['text'].progress_apply(classifica_com_gpt)

Classificando textos: 100%|█████████████████| 1500/1500 [19:05<00:00,  1.31it/s]


In [78]:
data

,text,prediction,pred_gpt4_turbo
0,Aquele momento que o gato preto volta pra não cruzar o seu caminho,1,ironia
1,"Marcelo Oliveira pode estar poupando Matheus Sales para encarar o Santos, de Lucas Lima.",1,n_ironia
2,ECONOMIAALERTANÃO! QUEIRAM ASSISTIR O MAIOR ESCOAMENTO FINANCEIRO MUNDIALQUEREMOS! UM BRASIL PARA TODOS!DiretasJÁ!,0,ironia
3,São Paulo caindo de pé como um bom perdedor.,1,ironia
4,claro que foi....,1,ironia
...,...,...,...
1495,interessante perceber como o catedrático vai dar uma entrevista precisamente ao jornal de quem o tanto se queixa,1,ironia
1496,So notícia boa! Uhuuu,1,ironia
1497,"AGORA VAI! Realmente bater em jogador resolve tudo, vasco não cai mais!",1,ironia
1498,Economia.Presidente do Senado admite possibilidade de acelerar reforma trabalhista,0,n_ironia


In [79]:
data.to_csv('teste_ironia_gpt4_turbo.csv', index=False)